# Metrics on scikit-learn github repo

In [ ]:
import json
import datetime
import numpy as np
import pandas as pd

In [ ]:
with open("../data/issues.json") as f:
    istr = f.read()

In [ ]:
ijson = json.loads(istr)
ntot = len(ijson)

In [ ]:
number = [ ijson[i]["node"]["number"] for i in range(0, ntot) ]
creatDate = [ ijson[i]["node"]["createdAt"] for i in range(0, ntot) ]
closDate = [ ijson[i]["node"]["closedAt"] for i in range(0, ntot) ]
state = [ ijson[i]["node"]["state"] for i in range(0, ntot) ]

In [ ]:
author = []
firstcommentedBy = []
firstcommentedAt = []
for i in range(0, ntot):
    auth = ijson[i]["node"]["author"]
    if (auth != None):
        author.append(auth["login"])
    else:
        author.append("")
    comments = ijson[i]["node"]["comments"]["edges"]
    if (len(comments) > 0):
        if (comments[0]["node"]["author"] != None):
            firstcommentedBy.append(comments[0]["node"]["author"]["login"])
        else:
            firstcommentedBy.append("")
        firstcommentedAt.append(comments[0]["node"]["createdAt"])
    else:    
        firstcommentedBy.append("")
        firstcommentedAt.append("")

In [ ]:
from datetime import datetime
from datetime import timedelta

crdate = []
cldate = []
answdate = []
duration = []
reaction = []
delta = timedelta(hours=1)

for i in range(0, ntot):
    crdate.append(pd.to_datetime(creatDate[i], format='%Y-%m-%dT%H:%M:%SZ'))
    cldate.append(pd.to_datetime(closDate[i], format='%Y-%m-%dT%H:%M:%SZ'))
    if closDate[i] != None:
        duration.append(pd.Timedelta.to_pytimedelta(cldate[i] - crdate[i]) / delta )
    else:
        duration.append(None)
    try:
        answdate.append(pd.to_datetime(firstcommentedAt[i], format='%Y-%m-%dT%H:%M:%SZ'))
        reaction.append(pd.Timedelta.to_pytimedelta(answdate[i] - crdate[i]) / delta )
    except:
        answdate.append(None)
        reaction.append(None)

In [ ]:
ntot

In [ ]:
idata = { "Number" : number, "Author" : author, "CreatedAt" : crdate, "ClosedAt" : cldate,
          "Duration" : duration, "FcommAuthor" : firstcommentedBy,
          "FcommDate" : answdate, "AnswerTime" : reaction }
idf = pd.DataFrame(idata)

In [ ]:
idf.head()

In [ ]:
import pandas_profiling
idf.profile_report()

In [ ]:
gidf = idf.groupby([idf['CreatedAt'].dt.to_period('Y')]).count().unstack()
gidf['CreatedAt'].plot(kind='bar', color='red')
gidf['ClosedAt'].plot(kind='bar', color='blue')
#gidf['FcommDate'].plot(kind='bar', color='green', alpha='0.5')

In [ ]:
yidf = idf.groupby([idf['CreatedAt'].dt.to_period('Y')])
yidf['AnswerTime'].hist()

In [ ]:
yidf['Duration'].hist()